#### 이미지 분류 예제(Gradio 사용)

In [1]:
# 사전설치 : pip install pillow
import gradio as gr
import tensorflow as tf
import numpy as np
from PIL import Image
import requests
from io import BytesIO

c:\Python310\aiproject\llm2_Study\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Tensorflow MobileNet V2 모델 로드
model = tf.keras.applications.MobileNetV2(weights="imagenet")

14536120/14536120 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [3]:
def predict_image(image_url):
    try:
        # URL에서 이미지 가져오기
        response = requests.get(image_url)
        image = Image.open(BytesIO(response.content)).resize((224, 224))  # BytesIO 사용하여 이미지 열기

        # 이미지를 배열로 변환
        image_array = tf.keras.preprocessing.image.img_to_array(image)
        image_array = tf.expand_dims(image_array, axis=0)   # 배치 차원 추가
        image_array = tf.keras.applications.mobilenet_v2.preprocess_input(image_array)  # 전처리
        
        # 예측 수행
        predictions = model.predict(image_array)
        decoded_predictions = tf.keras.applications.mobilenet_v2.decode_predictions(predictions, top=3)[0]
        
        # Gradio Label 컴포넌트에 맞게 결과 형식 변경
        # { label: confidence1, label2: confidence2, ...} 형식으로 변환
        result = {label: float(prob) for (_, label, prob) in decoded_predictions}
        return result
    
    except Exception as e:
        return {"error": 1.0}   # 에러 발생 시 기본값 반환

In [4]:
# Gradio 인터페이스 생성
iface = gr.Interface(
    fn=predict_image,
    inputs=gr.Textbox(label="이미지  URL 입력"),
    outputs=gr.Label(num_top_classes=3, label="예측 결과"),
    title="음식 이미지 분류",
    description="이미지 URL을 입력하면 상위 3개의 예측 결과를 확률과 함께 표시합니다."
)

In [5]:
# 인터페이스 실행
iface.launch(server_port=7861, server_name="0.0.0.0", debug=True)

# 예시 이미지 URL : https://health.chosun.com/site/data/img_dir/2024/04/19/2024041901914_0.jpg

* Running on local URL:  http://0.0.0.0:7861

To create a public link, set `share=True` in `launch()`.


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 555ms/step
35363/35363 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Created dataset file at: .gradio\flagged\dataset1.csv
Keyboard interruption in main thread... closing server.


In [6]:
iface.close()

Closing server running on port: 7861
